In [4]:
import pickle

# Load the pickle file
with open('/home/vit/Projects/flexibility-analysis/datasets/scPDB/scPDB_filt_df.pkl', 'rb') as file:
    scPDB_filt_df = pickle.load(file)

# Display the loaded data (optional)
print(scPDB_filt_df)

         dataset      ID    chains n_chains n_ress   ligs  n_ligs  \
0     scPDB_filt  2zb4_1     [{A}]      [1]   [21]  [5OP]       1   
1     scPDB_filt  4yfy_4     [{B}]      [1]   [31]  [1YJ]       1   
2     scPDB_filt  4lt6_2     [{B}]      [1]   [33]  [3AT]       1   
3     scPDB_filt  5mdh_1     [{A}]      [1]   [44]  [NAD]       1   
4     scPDB_filt  2w0w_1     [{A}]      [1]   [42]  [LZS]       1   
...          ...     ...       ...      ...    ...    ...     ...   
5015  scPDB_filt  2fxe_1  [{B, A}]      [2]   [42]  [DR7]       1   
5016  scPDB_filt  4ckc_2     [{D}]      [1]   [38]  [SAH]       1   
5017  scPDB_filt  2giu_1     [{A}]      [1]   [32]  [FBR]       1   
5018  scPDB_filt  1ha3_2  [{B, A}]      [2]   [45]  [MAU]       1   
5019  scPDB_filt  3sai_2     [{B}]      [1]   [28]  [9DR]       1   

              chain_lens  
0             {'A': 351}  
1             {'B': 241}  
2             {'B': 462}  
3             {'A': 333}  
4             {'A': 451}  
...      

In [5]:
with open('/home/vit/Projects/flexibility-analysis/datasets/LIGYSIS/LIGYSIS_human_chains_per_lig_MAY_2024.pkl', 'rb') as file:
    LIGYSIS_human_chains_per_lig_MAY_2024 = pickle.load(file)

print(LIGYSIS_human_chains_per_lig_MAY_2024.keys())
# Display the loaded data (optional)
print(LIGYSIS_human_chains_per_lig_MAY_2024['A0A0B4J272_1'])

dict_keys(['A0A0B4J272_1', 'A0AVT1_1', 'A0FGR8_1', 'A1L3X0_1', 'A1Z1Q3_1', 'A2RUC4_1', 'A4D126_1', 'A4D2B0_1', 'A6NHR9_1', 'A6NIH7_1', 'A6NK06_1', 'A8K7I4_1', 'A8K979_1', 'A8MT69_1', 'A8MUM7_1', 'A8MW92_1', 'B2RXH2_1', 'O00139_1', 'O00141_1', 'O00151_2', 'O00159_1', 'O00167_1', 'O00180_1', 'O00182_1', 'O00182_2', 'O00187_1', 'O00194_1', 'O00204_1', 'O00206_1', 'O00212_1', 'O00213_2', 'O00214_1', 'O00217_1', 'O00222_1', 'O00238_1', 'O00244_1', 'O00264_1', 'O00294_1', 'O00299_1', 'O00311_1', 'O00329_1', 'O00338_1', 'O00370_2', 'O00391_2', 'O00408_1', 'O00409_1', 'O00411_1', 'O00429_1', 'O00443_2', 'O00444_1', 'O00445_1', 'O00463_1', 'O00478_2', 'O00481_1', 'O00481_2', 'O00482_1', 'O00482_2', 'O00488_1', 'O00506_1', 'O00522_2', 'O00548_1', 'O00560_1', 'O00571_1', 'O00602_1', 'O00625_1', 'O00757_1', 'O00763_1', 'O00763_3', 'O00764_1', 'O00767_1', 'O14497_1', 'O14520_1', 'O14545_1', 'O14556_1', 'O14561_1', 'O14595_1', 'O14607_1', 'O14618_1', 'O14618_2', 'O14638_1', 'O14640_1', 'O14649_1', '

In [6]:
with open('/home/vit/Projects/flexibility-analysis/datasets/LIGYSIS/LIGYSIS_sites_DEF_TRANS.pkl', 'rb') as file:
    LIGYSIS_human_chains_per_lig_MAY_2024 = pickle.load(file)
import pandas
df = pandas.DataFrame(LIGYSIS_human_chains_per_lig_MAY_2024)
# Display the loaded data (optional)
print(df.head(10))

      ACC SEG  ID                                             up_aas  n_aas  \
0  P03372   2   0  [339, 341, 342, 343, 344, 345, 346, 347, 349, ...     61   
1  P03372   2   1  [329, 330, 331, 332, 333, 335, 336, 337, 338, ...     18   
2  P03372   2   2                          [326, 327, 328, 330, 352]      5   
3  P00750   3   0  [342, 357, 358, 401, 402, 403, 482, 484, 485, ...     27   
4  P02100   1   0  [32, 39, 42, 43, 46, 60, 64, 67, 68, 71, 72, 8...     22   
5  P09210   1   0  [9, 14, 15, 41, 45, 53, 54, 55, 56, 66, 67, 68...     19   
6  P09210   1   1  [9, 12, 14, 15, 107, 108, 111, 112, 216, 220, ...     11   
7  P12429   1   0                       [31, 32, 33, 34, 35, 36, 75]      7   
8  P12429   1   1                [103, 104, 105, 106, 107, 108, 147]      7   
9  P12429   1   2      [186, 187, 189, 190, 191, 192, 193, 194, 231]      9   

     SEG_ID   SITE_NAME pdb_id auth_asym_id struct_asym_id rep_chain    RoG  \
0  P03372_2  P03372_2_0   1a52            A        

In [7]:
index = 4

assert len(df.iloc[index]['aas']) == len(df.iloc[index]['up_aas'])
"+".join([str(i) for i in df.iloc[index]['aas']])

'31+38+41+42+45+59+63+66+67+70+71+85+88+91+92+96+98+102+103+106+137+141'

In [8]:
df['protein_id'] = df['pdb_id'] + df['struct_asym_id']

In [23]:
import biotite.database.rcsb as rcsb
import biotite.structure.io.pdbx as pdbx
import biotite.structure as struc
import biotite.sequence as seq
import numpy as np
import os, sys
sys.path.append('../../utils/')
import constants

mapping = {'Aba': 'A', 'Ace': 'X', 'Acr': 'X', 'Ala': 'A', 'Aly': 'K', 'Arg': 'R', 'Asn': 'N', 'Asp': 'D', 'Cas': 'C',
           'Ccs': 'C', 'Cme': 'C', 'Csd': 'C', 'Cso': 'C', 'Csx': 'C', 'Cys': 'C', 'Dal': 'A', 'Dbb': 'T', 'Dbu': 'T',
           'Dha': 'S', 'Gln': 'Q', 'Glu': 'E', 'Gly': 'G', 'Glz': 'G', 'His': 'H', 'Hse': 'S', 'Ile': 'I', 'Leu': 'L',
           'Llp': 'K', 'Lys': 'K', 'Men': 'N', 'Met': 'M', 'Mly': 'K', 'Mse': 'M', 'Nh2': 'X', 'Nle': 'L', 'Ocs': 'C',
           'Pca': 'E', 'Phe': 'F', 'Pro': 'P', 'Ptr': 'Y', 'Sep': 'S', 'Ser': 'S', 'Thr': 'T', 'Tih': 'A', 'Tpo': 'T',
           'Trp': 'W', 'Tyr': 'Y', 'Unk': 'X', 'Val': 'V', 'Ycm': 'C', 'Sec': 'U', 'Pyl': 'O', 'Mhs': 'H', 'Snm': 'S',
           'Mis': 'S', 'Seb': 'S', 'Hic' : 'H', 'Fme': 'M', 'Asb': 'D', 'Sah': 'C', 'Smc': 'C', 'Tpq': 'Y', 'Onl': 'X',
           'Tox': 'W', '5x8': 'X', 'Ddz': 'A', 'Nep': 'X', 'T8l': 'T', 'Cgu': 'E', 'Caf': 'C', 'Dsn': 'S', 'Tys': 'Y',
           'Als': 'S', 'Kcx': 'K', 'P1l': 'C', 'Fgp': 'X', 'Scy': 'C', 'Cxm': 'M', 'Css': 'C', 'Led': 'L', 'Cgl': 'C',
           'Mlz': 'K', 'Scs': 'A'}


def get_sequence(protein_id):
    mmcif_filepath = f'{constants.CIF_FILES_PATH}/{protein_id[:4]}.cif'

    if not os.path.isfile(mmcif_filepath):
        rcsb.fetch(protein_id[:4], 'cif', target_path=constants.CIF_FILES_PATH)

    mmcif_file = pdbx.CIFFile.read(mmcif_filepath) # reads the entity_poly.pdbx_seq_one_letter_code_can entry.

    chain_mask = mmcif_file.block['pdbx_poly_seq_scheme']['asym_id'].as_array() == protein_id[4:]
    sequence = mmcif_file.block['pdbx_poly_seq_scheme']['mon_id'].as_array()[chain_mask]
    sequence = ''.join([mapping[i.capitalize()] for i in sequence])

    return sequence

df.groupby('ACC')
for accession_id, sequence_group in df.groupby('ACC'):
    # find largest contributor:
    largest_pocket = []
    largest_protein_id = None
    for protein_id, protein_group in sequence_group.groupby('protein_id'):
        if len(set(sum(protein_group['aas'].values.tolist(), []))) > len(largest_pocket):
            largest_pocket = set(sum(protein_group['aas'].values.tolist(), []))
            largest_protein_id = protein_id
    
    if os.path.isfile(f'/home/vit/Projects/flexibility-analysis/datasets/LIGYSIS/sequences/{largest_protein_id}.fasta'):
        continue

    print(f'processing {largest_protein_id}')
    
    sequence = get_sequence(largest_protein_id)

    largest_pocket = [int(''.join(filter(str.isdigit, i))) for i in largest_pocket]
    assert max(largest_pocket) <= len(sequence), f'{max(largest_pocket)} <= {len(sequence)}'
    assert min(largest_pocket) > 0

    with open(f'/home/vit/Projects/flexibility-analysis/datasets/LIGYSIS/sequences/{largest_protein_id}.fasta', 'w') as f:
        f.write(f'>{largest_protein_id}\n')
        f.write(str(sequence))
    np.save(f'/home/vit/Projects/flexibility-analysis/datasets/LIGYSIS/indices/{largest_protein_id}.npy', np.array(list(largest_pocket)))


processing 2cmwA
processing 2lobA
processing 4zdpA
processing 5i0iB
processing 1k9iH
processing 8bzyA
processing 2crwA
processing 1x3sA
processing 7tzbB
processing 7cocA
processing 2ocbA
processing 3f1rB
processing 1smoB
processing 4mumA
processing 5ngzA
processing 7qbeD
processing 6gwjC
processing 4iejA
processing 1oj6D
processing 6ppaA
processing 3f5oE
processing 2vhfA
processing 5x9bA
processing 5x49A
processing 2q3fB
processing 7d1zJ
processing 4fomA
processing 3gbjA
processing 2c30A
processing 3rayA
processing 5eh2E
processing 3ctzA
processing 4zg4A
processing 6l5mC
processing 2gaoB
processing 7y0iA
processing 5u6hA
processing 5mvxA
processing 4wgkA
processing 7dtaA
processing 5wyxA
processing 2gzvA
processing 7miiD
processing 2jbhB
processing 4yndA
processing 4b4oG
processing 2c43A
processing 6h2vA
processing 6s7oH
processing 6v5cA
processing 2e9sA
processing 3vm8A
processing 3c5hA
processing 2cq9A
processing 7w6sC
processing 6wq1B
processing 6zphB
processing 2y43B
processing 5i3

## Frustrace 
1. male pockety vs velke pockety NEBO SPIS normovat pocet frustrovanych residui/paru a vydelit poctem residui v pocketu
2. procento frustrace v pocketu vs. venku vs rigid vs cryptic (jaka mira frustrace je v rigid vs cryptic)
3. kolik rigid vs cryptic ma NEJAKOU frustraci a kolik ZADNOU 
4. spocitat pocet apo-holo distances (prihodit do grafu)
5. prepocitat na frustraevo: https://hub.docker.com/r/proteinphysiologylab/frustraevo